In [87]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import uncertainties as unc
import scipy
from scipy import stats
from scipy.stats import norm 
from scipy.stats import uniform
from scipy.stats import binom
from scipy.stats import poisson
from scipy.special import factorial
from numpy import random
import scipy.constants as const
from tabulate import tabulate
import iminuit.minimize as minimize
import math
from tqdm import tqdm
%matplotlib inline

In [88]:
# 1. We require signal event s>0 for positive signal yield.
#        Therefore, the test statistics q_0 is 0 if N_obs <= Nb
# 2. Compute two Poisson loglikelihood of
#   a) backgorund only model
#   b) signal+background model
#   Evaluate -2 log likelihood ratio between a) and b)

def q02(N_obs, Nb):
    if N_obs <= Nb:
        q0_out = 0
    else:
        q0_out = -2 * np.log(poisson.pmf(N_obs,Nb) / poisson.pmf(N_obs,N_obs))
    return q0_out

In [89]:
def FreqnetistZ0(N_obs,N_b):
    #set random seed to guarantee reproducibility
    np.random.seed(seed=8)
    q0_toy = 0
    # Step1. Generate f(q_0|B) distribtuion
    # Generate toy experiments based on background only model and compute q_0␣↪for each experiment
    q_dist = [] # Initialize array to fill with distribution values for f(q_0|B)
    N_toy = 10000 # Generate 1000 points
    for _ in tqdm(range(N_toy)):
        toy = np.random.poisson(N_b)
        q0_toy = q02(toy, N_b)
        q_dist.append(q0_toy)
    q_dist = np.array(q_dist)
    # Step2. Compute p-value. The fraction of toy experiments with q_0 greater␣↪or eqal to q_{0,obs}
    q0_obs = q02(N_obs, N_b) # Find q0 from given Nb and N_obs
    pvalue = np.sum(q_dist >= q0_obs) / N_toy # p-value
    Zscore= scipy.stats.norm.ppf(1 - pvalue)
    return Zscore

Now, let’s apply our code for numerical calculations.
Consider the case that backogrund only model with yields b=0.5 and observed events n=5.
Calclate discovery significance.

In [90]:
Nobs=5
Nb=0.5
F = FreqnetistZ0(Nobs,Nb)
# Write your code here
print("Z0freq:%.2f"%F)



100%|██████████| 10000/10000 [00:00<00:00, 20075.24it/s]

Z0freq:3.72


    Q1: How is the number compared to the Baysian signficance from homework 5? 
    From homework 5, the Bayesian significance was 4.18, which is greater than the Frequentist significance found here.

Consdier a background only model with yields b=4 and observed events n=5. Calculate Baysian signifiance and Frequqnt signficance, and compare the results.

In [91]:
Nobs=5
Nb=4

# Frequentist sig
F2 = FreqnetistZ0(Nobs,Nb)
print("Z0freq:%.2f"%F2)

# Bayesian sig using funcgtion from HW 5
def BayesianZ0(N_obs,N_b):
    pvalue = 1-poisson.cdf(N_obs, N_b)
    Zscore= scipy.stats.norm.ppf(1-pvalue)
    return Zscore

B2 = BayesianZ0(Nobs, Nb)
print("Z0Bayesian:%.2f"%B2)


100%|██████████| 10000/10000 [00:00<00:00, 22470.75it/s]

Z0freq:0.34
Z0Bayesian:0.79


For a higher background yield, the ratio of Bayesian significance to the Frequentist significance is much larger than before where the background yield was much smaller. This is because in the frequentist calculation, the toy values only take in the background and here Nb is so much closer to Nobserved the frequentist significance is much smaller by comparison. 